In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
TEST = False

Phase_1 = False
Phase_2 = False
Phase_3 = False
Phase_4 = False

DATA_FILE = "sc2021_train_deals.csv"
NAME_OUTPUT_FILE = 'A_XGBM-tuning(v49)'
NAME_OUTPUT_FOR_STAKING = 'XGBM_probs_for_staking(v49).npy'
NAME_OUTPUT_FOR_STAKING_TEST = 'XGBM_probs-test_for_staking(v49).npy'
TARGET = "target"


# Range of values for selection in phase 1 :
import numpy as np
N_ESTIMATORS_RANG = list(range(10,500,1))#  [50000] #[5000] #[500] #
RANG_LEARNING_RATE =   list(np.arange(0.01,0.1,0.005)) # 0.0015 #list(np.arange(5e-3,1e-1,5e-3)) # [0.005]

N_TRIALS = 50

SEED = 82736

SELECT_PRED = False
SELECT_PRED2 = False

HIGH_PARAM_END = False

# ------------------------------------------------------------------------------
# Path definition
# ------------------------------------------------------------------------------
#from pathlib import Path
import pathlib
#input_path = Path('Data')
DATA_DIR = pathlib.Path(".")

AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]

LOG_PATH = pathlib.Path("./log/")
NAME_RUN_LOG = 'Run__Logs.log'

In [3]:
# Берем полученные наилучшие значения гиперпараметров
# xgb.XGBRegressor() с прошлых итераций
params = {
         'n_estimators': 330,
         'learning_rate': 0.055, 
        'random_state': SEED,
        'max_depth': 4, #3, 
        'min_child_weight': 115, #30, 
        'colsample_bytree': 0.7359281520613952, 
        'subsample': 0.3350726311191104, 
        'lambda': 0.07017450086376546, 
        'alpha': 0.06778367298243107,
        'gamma': 0.30129207712879835
        }
#Наилучшие значения гиперпараметров: {'ml__colsample_bytree': 0.7359281520613952, 'ml__subsample': 0.3350726311191104}
#Наилучшее значение AUC: -0.626
#Наилучшие значения гиперпараметров: {'ml__lambda': 0.07017450086376546, 'ml__alpha': 0.06778367298243107, 'ml__gamma': 0.30129207712879835}
#Наилучшее значение AUC: -0.625

In [4]:
from time import time
import datetime
start_time = time()


    
LOG_PATH.mkdir(parents=True, exist_ok=True)

with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
    print('\n\n***********************\n***********************', 
          file=logfile)
    print('Время старта скрипта: {}'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)
print('Время старта скрипта: {}'.format(datetime.datetime.now().strftime("%d-%m-%Y %H:%M")))
    
#params

Время старта скрипта: 17-12-2021 19:22


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error, make_scorer

import numpy as np
# импортируем классы KFold, TimeSeriesSplit и GroupKFold,
# реализующие стратегии перекрестной проверки, и
# класс GridSearchCV для поиска гиперпараметров
from sklearn.model_selection import (
    KFold,
    TimeSeriesSplit, 
    GroupKFold,
    GridSearchCV
)
from tqdm.notebook import tqdm
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder, OrdinalEncoder, LeaveOneOutEncoder, CatBoostEncoder
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, GradientBoostingRegressor
import dill

warnings.simplefilter(action='ignore', category=FutureWarning)
# увеличиваем количество отображаемых столбцов
pd.set_option('display.max_columns', 50)

In [6]:
#!pip install -U scikit-learn --user

In [7]:
!conda activate sibur

In [8]:
import sklearn
print (sklearn.__version__)

1.0.1


In [9]:
#!pip install optuna

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import optuna
from optuna.samplers import RandomSampler
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, UniformDistribution
from sklearn.model_selection import cross_val_predict

In [11]:
# увеличиваем количество выводимых столбцов
pd.set_option('display.max_columns', 60)
# загружаем набор
data = pd.read_csv(DATA_DIR.joinpath(DATA_FILE), parse_dates=["month", "date"])

In [12]:
group_ts = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
group_ts

month                                                                  2018-01-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              340.0   
133           0            Белоруссия Минская обл.       10942                0.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942                0.0   
                           Казахстан  г. Нур-Султан      13301                0.0   
...                                                                           ...   
986           9943         Россия     Смоленская обл.    17460                0.0   
998           0            Россия     Ленинградская обл. 18079                0.0   
              3380         Россия     Ленинградская обл. 14956                0.0   
              5410         Россия     г. Санкт-Петербург 14956               60.0   
              6346         Россия     Респ. Башкортостан 10737                0.0   

month                                                                  2018-02-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              340.0   
133           0            Белоруссия Минская обл.       10942                0.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942               20.0   
                           Казахстан  г. Нур-Султан      13301                0.0   
...                                                                           ...   
986           9943         Россия     Смоленская обл.    17460                0.0   
998           0            Россия     Ленинградская обл. 18079                0.0   
              3380         Россия     Ленинградская обл. 14956                0.0   
              5410         Россия     г. Санкт-Петербург 14956               60.0   
              6346         Россия     Респ. Башкортостан 10737                0.0   

month                                                                  2018-03-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              260.0   
133           0            Белоруссия Минская обл.       10942                0.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942                0.0   
                           Казахстан  г. Нур-Султан      13301               30.0   
...                                                                           ...   
986           9943         Россия     Смоленская обл.    17460                0.0   
998           0            Россия     Ленинградская обл. 18079                0.0   
              3380         Россия     Ленинградская обл. 14956                0.0   
              5410         Россия     г. Санкт-Петербург 14956              100.0   
              6346         Россия     Респ. Башкортостан 10737                0.0   

month                                                                  2018-04-01  \
material_code company_code country    region             manager_code               
124           7278         Россия     Респ. Татарстан    17460              240.0   
133           0            Белоруссия Минская обл.       10942              200.0   
                                      Могилевская обл.   10942                0.0   
                                      г. Минск           10942                0.0   
                           Казахстан  г. Нур-Султан      13301               30.0   
...                                                                           ...   
98

In [13]:
def get_features_Cv(df: pd.DataFrame, month: pd.Timestamp, N=6, MDAD_1=2,  MNGR_GRP_MDAD=0) -> pd.DataFrame:
    """Calculate features for `month`."""

    start_period = month - pd.offsets.MonthBegin(N)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    # формируем лаги за N месяцев
    features[[f"vol_tm{i}" for i in range(N, 0, -1)]] = df.loc[:, start_period:end_period].copy()

    ### !!!!!!!!!!!!!!   #################################################################################
    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    
    # Добавление ГРУППОВЫХ скользящих средних абсолютные отклонения(MDAD)
    if MNGR_GRP_MDAD != 0:
        period = MNGR_GRP_MDAD
        df2 = df.copy()
        df2[df2.columns.to_list()] = \
                                df2.groupby(level='manager_code').transform(lambda x: x.mean())
        grp_manager_roll_mean = df2.rolling(period, axis=1, min_periods=1)
        features = \
        features.join(grp_manager_roll_mean.apply(lambda x: np.nanmedian(np.abs(x - np.nanmedian(x))) , raw=True 
                        ).iloc[:, -1].rename("MNGR_GRP_MDAD"+str(period))) 
    
    if MDAD_1 != 0:
        period = MDAD_1
        rolling = df.rolling(period, axis=1, min_periods=1)
        features = features.join( 
                        rolling.apply(lambda x: np.nanmedian(np.abs(x - np.nanmedian(x))) , raw=True 
                        ).iloc[:, -1].rename("mdad_"+str(period)) )
    '''
    if MDAD_2 != 0 and MDAD_2 != MDAD_1:
        period = MDAD_2
        rolling = df.rolling(MDAD_2, axis=1, min_periods=1)
        features = features.join( 
                        rolling.apply(lambda x: np.nanmedian(np.abs(x - np.nanmedian(x))) , raw=True 
                        ).iloc[:, -1].rename("mdad_"+str(period)) )    
                        
    ### !!!!!!!!!!!!!!   ##################################################################################
    # Cv1
    
    
    
    ##################################################################################################
    '''
    ##################################################################################################
    ##################################################################################################

    # выделяем КВАРТАЛЫ
    #features["quarter"] = month.quarter

    return features

In [14]:
cv_traine_range = pd.date_range("2019-01-01", "2020-07-01", freq="MS")

#cv_traine_range = pd.date_range("2018-07-01", "2020-07-01", freq="MS")
cv_traine_range, len(cv_traine_range)

(DatetimeIndex(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
                '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
                '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
                '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
                '2020-05-01', '2020-06-01', '2020-07-01'],
               dtype='datetime64[ns]', freq='MS'),
 19)

In [15]:
%%time

full_features2 = {}

dataset_features2 = []
for target_month in tqdm(cv_traine_range):
#################################################################################################################################

    features2 = get_features_Cv(group_ts, target_month, MDAD_1=2, MNGR_GRP_MDAD=0)

#################################################################################################################################
    features2[TARGET] = group_ts[target_month]
    dataset_features2.append(features2.reset_index())
full_features2 = pd.concat(dataset_features2, ignore_index=True)

CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]
# создаем список  переменных
FTS_COLS = full_features2.columns.tolist()
FTS_COLS.remove(TARGET)
#################################################################################################################################
print(FTS_COLS)

  0%|          | 0/19 [00:00<?, ?it/s]

['material_code', 'company_code', 'country', 'region', 'manager_code', 'month', 'vol_tm6', 'vol_tm5', 'vol_tm4', 'vol_tm3', 'vol_tm2', 'vol_tm1', 'last_year_avg', 'mdad_2']
Wall time: 34.2 s


In [16]:
#cv_features = full_features2[:-941]
cv_features = full_features2.copy() #[941:]

for c in cv_features.columns:
    col_type = cv_features[c].dtype
    if col_type == 'object': # or col_type == 'int64': # or col_type.name == 'category':
        cv_features[c] = cv_features[c].astype('category')

cv_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17879 entries, 0 to 17878
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   material_code  17879 non-null  int64   
 1   company_code   17879 non-null  int64   
 2   country        17879 non-null  category
 3   region         17879 non-null  category
 4   manager_code   17879 non-null  int64   
 5   month          17879 non-null  int64   
 6   vol_tm6        17879 non-null  float64 
 7   vol_tm5        17879 non-null  float64 
 8   vol_tm4        17879 non-null  float64 
 9   vol_tm3        17879 non-null  float64 
 10  vol_tm2        17879 non-null  float64 
 11  vol_tm1        17879 non-null  float64 
 12  last_year_avg  17879 non-null  float64 
 13  mdad_2         17879 non-null  float64 
 14  target         17879 non-null  float64 
dtypes: category(2), float64(9), int64(4)
memory usage: 1.8 MB


In [17]:
cv_features

,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,mdad_2,target
0,124,7278,Россия,Респ. Татарстан,17460,1,220.0,220.0,220.0,280.0,280.0,280.0,260.000000,0.0,200.0
1,133,0,Белоруссия,Минская обл.,10942,1,0.0,0.0,0.0,0.0,0.0,0.0,21.666667,0.0,0.0
2,133,0,Белоруссия,Могилевская обл.,10942,1,0.0,0.0,100.0,220.0,20.0,0.0,40.000000,10.0,0.0
3,133,0,Белоруссия,г. Минск,10942,1,180.0,99.0,60.0,400.0,120.0,20.0,91.583333,50.0,40.0
4,133,0,Казахстан,г. Нур-Султан,13301,1,40.0,20.0,40.0,30.0,0.0,40.0,19.166667,20.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17874,986,9943,Россия,Смоленская обл.,17460,7,0.0,21.0,63.0,125.0,84.0,84.0,31.416667,0.0,83.0
17875,998,0,Россия,Ленинградская обл.,18079,7,5.0,5.0,5.0,0.0,3.0,3.0,4.750000,0.0,9.0
17876,998,3380,Россия,Ленинградская обл.,14956,7,73.0,74.0,122.0,100.0,15.0,30.0,85.583333,7.5,50.0
17877,998,5410,Россия,г. Санкт-Петербург,14956,7,100.0,180.0,180.0,100.0,140.0,40.0,118.333333,50.0,0.0


In [18]:
# 18 групп - блоки по 1 наблюдению(в 1 наблюдении данные 941 временного ряда)
N_timeseries = 941 # в 1 наблюдении данные 941 временного ряда
start_train = 1  # 

test_size=N_timeseries #*3
#max_train_size = N_timeseries*6
n_splits= 17 #int(len(full_features2)/(test_size))-1 - start_train # 

tscv = TimeSeriesSplit(n_splits=n_splits, 
                       #max_train_size=max_train_size, 
                       test_size=test_size)

verbose =  False # True # 

In [19]:
for train_index, test_index in tscv.split(cv_features): # tscv.split(X): # tscv.split(cv_features)
    print("TRAIN: ", train_index[0],"...",train_index[-1])
    print("TEST: ", test_index[0],"...",test_index[-1])

    X_train, X_test = cv_features.iloc[train_index], cv_features.iloc[test_index]
    y_train, y_test = cv_features.iloc[train_index][TARGET], cv_features.iloc[test_index][TARGET]
    
    print('Обучающий набор: %f' % (len(X_train)/941)) #test_size))
    print('Тестовый набор: %d' % (len(X_test)/941)) #test_size))

    #display(X_train)
    display(X_test)
print("tscv.get_n_splits()=",tscv.get_n_splits())

TRAIN:  0 ... 1881
TEST:  1882 ... 2822
Обучающий набор: 2.000000
Тестовый набор: 1


,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,mdad_2,target
1882,124,7278,Россия,Респ. Татарстан,17460,3,220.0,280.0,280.0,280.0,200.0,200.0,236.666667,0.0,200.0
1883,133,0,Белоруссия,Минская обл.,10942,3,0.0,0.0,0.0,0.0,0.0,0.0,21.666667,0.0,0.0
1884,133,0,Белоруссия,Могилевская обл.,10942,3,100.0,220.0,20.0,0.0,0.0,80.0,46.666667,40.0,142.0
1885,133,0,Белоруссия,г. Минск,10942,3,60.0,400.0,120.0,20.0,40.0,0.0,93.250000,20.0,0.0
1886,133,0,Казахстан,г. Нур-Султан,13301,3,40.0,30.0,0.0,40.0,40.0,50.0,26.666667,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,986,9943,Россия,Смоленская обл.,17460,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2819,998,0,Россия,Ленинградская обл.,18079,3,5.0,5.0,0.0,0.0,5.0,5.0,2.000000,0.0,0.0
2820,998,3380,Россия,Ленинградская обл.,14956,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,80.0
2821,998,5410,Россия,г. Санкт-Петербург,14956,3,80.0,80.0,80.0,100.0,120.0,119.0,88.250000,0.5,160.0


TRAIN:  0 ... 2822
TEST:  2823 ... 3763
Обучающий набор: 3.000000
Тестовый набор: 1


,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,mdad_2,target
2823,124,7278,Россия,Респ. Татарстан,17460,4,280.0,280.0,280.0,200.0,200.0,200.0,231.666667,0.0,185.0
2824,133,0,Белоруссия,Минская обл.,10942,4,0.0,0.0,0.0,0.0,0.0,0.0,21.666667,0.0,36.0
2825,133,0,Белоруссия,Могилевская обл.,10942,4,220.0,20.0,0.0,0.0,80.0,142.0,58.500000,31.0,103.0
2826,133,0,Белоруссия,г. Минск,10942,4,400.0,120.0,20.0,40.0,0.0,0.0,93.250000,0.0,0.0
2827,133,0,Казахстан,г. Нур-Султан,13301,4,30.0,0.0,40.0,40.0,50.0,0.0,24.166667,25.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,986,9943,Россия,Смоленская обл.,17460,4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3760,998,0,Россия,Ленинградская обл.,18079,4,5.0,0.0,0.0,5.0,5.0,0.0,2.000000,2.5,8.0
3761,998,3380,Россия,Ленинградская обл.,14956,4,0.0,0.0,0.0,0.0,0.0,80.0,6.666667,40.0,94.0
3762,998,5410,Россия,г. Санкт-Петербург,14956,4,80.0,80.0,100.0,120.0,119.0,160.0,93.250000,20.5,120.0


TRAIN:  0 ... 3763
TEST:  3764 ... 4704
Обучающий набор: 4.000000
Тестовый набор: 1


...

TRAIN:  0 ... 15996
TEST:  15997 ... 16937
Обучающий набор: 17.000000
Тестовый набор: 1


,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,mdad_2,target
15997,124,7278,Россия,Респ. Татарстан,17460,6,0.0,0.0,0.0,0.0,0.0,0.0,5.166667,0.0,0.0
15998,133,0,Белоруссия,Минская обл.,10942,6,207.0,17.0,72.0,250.0,394.0,288.0,169.166667,53.0,210.0
15999,133,0,Белоруссия,Могилевская обл.,10942,6,0.0,0.0,0.0,0.0,166.0,62.0,19.000000,52.0,0.0
16000,133,0,Белоруссия,г. Минск,10942,6,0.0,0.0,0.0,0.0,21.0,0.0,22.416667,10.5,0.0
16001,133,0,Казахстан,г. Нур-Султан,13301,6,45.0,0.0,50.0,40.0,0.0,0.0,29.166667,0.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16933,986,9943,Россия,Смоленская обл.,17460,6,0.0,0.0,21.0,63.0,125.0,84.0,24.416667,20.5,84.0
16934,998,0,Россия,Ленинградская обл.,18079,6,6.0,5.0,5.0,5.0,0.0,3.0,5.333333,1.5,3.0
16935,998,3380,Россия,Ленинградская обл.,14956,6,29.0,73.0,74.0,122.0,100.0,15.0,93.166667,42.5,30.0
16936,998,5410,Россия,г. Санкт-Петербург,14956,6,100.0,100.0,180.0,180.0,100.0,140.0,123.333333,20.0,40.0


TRAIN:  0 ... 16937
TEST:  16938 ... 17878
Обучающий набор: 18.000000
Тестовый набор: 1


,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,mdad_2,target
16938,124,7278,Россия,Респ. Татарстан,17460,7,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
16939,133,0,Белоруссия,Минская обл.,10942,7,17.0,72.0,250.0,394.0,288.0,210.0,179.833333,39.0,249.0
16940,133,0,Белоруссия,Могилевская обл.,10942,7,0.0,0.0,0.0,166.0,62.0,0.0,19.000000,31.0,0.0
16941,133,0,Белоруссия,г. Минск,10942,7,0.0,0.0,0.0,21.0,0.0,0.0,22.416667,0.0,6.0
16942,133,0,Казахстан,г. Нур-Султан,13301,7,0.0,50.0,40.0,0.0,0.0,50.0,30.000000,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17874,986,9943,Россия,Смоленская обл.,17460,7,0.0,21.0,63.0,125.0,84.0,84.0,31.416667,0.0,83.0
17875,998,0,Россия,Ленинградская обл.,18079,7,5.0,5.0,5.0,0.0,3.0,3.0,4.750000,0.0,9.0
17876,998,3380,Россия,Ленинградская обл.,14956,7,73.0,74.0,122.0,100.0,15.0,30.0,85.583333,7.5,50.0
17877,998,5410,Россия,г. Санкт-Петербург,14956,7,100.0,180.0,180.0,100.0,140.0,40.0,118.333333,50.0,0.0


tscv.get_n_splits()= 17


In [20]:
# создаем списки категориальных признаков
#cat_columns = data.select_dtypes(include='object').columns
cat_columns = CAT_COLS

# создаем список количественных признаков
#num_columns = train.select_dtypes(exclude='object').columns.tolist()
num_columns =  [x for x in cv_features.columns.to_list() if x not in CAT_COLS + [TARGET]]
print(cat_columns)
print(num_columns)

['material_code', 'company_code', 'country', 'region', 'manager_code', 'month']
['vol_tm6', 'vol_tm5', 'vol_tm4', 'vol_tm3', 'vol_tm2', 'vol_tm1', 'last_year_avg', 'mdad_2']


> # ********  Phase 1 *************

In [21]:
# создаем конвейеры
cat_pipe = Pipeline([('enc', 
                      TargetEncoder(),
                      #OneHotEncoder(sparse=False, handle_unknown='ignore') 
                     )])
num_pipe = Pipeline([('scal', None)]) 

In [22]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [23]:
if Phase_1:
    params2 = params   
    # задаем итоговый конвейер
    ml_pipe = Pipeline([('tr', transformer), 
                        ('ml', xgb.XGBRegressor(**params) )])

In [24]:
N_ESTIMATORS_RANG = list(range(10,500,1))#  [50000] #[5000] #[500] #
RANG_LEARNING_RATE =   list(np.arange(0.01,0.1,0.005)) # 0.0015 #list(np.arange(5e-3,1e-1,5e-3)) # [0.005]

if Phase_1:
    # задаем сетку гиперпараметров
    param_grid = [{
                'tr__cat__enc': [LabelEncoder(), 
                                 OrdinalEncoder(),
                                 OneHotEncoder(sparse=False, handle_unknown='ignore'),
                                 TargetEncoder(),
                                 CatBoostEncoder(),
                                 LeaveOneOutEncoder()
                                ],
                'tr__num__scal': [# StandardScaler(),
                                  None
                                 ],
                'ml__learning_rate': RANG_LEARNING_RATE,
                'ml__n_estimators' : N_ESTIMATORS_RANG
    }]
    
    if TEST:
            param_grid = [{
                    'tr__cat__enc': [#TargetEncoder(),
                                     LabelEncoder(), 
                                     #OrdinalEncoder(),
                                     #OneHotEncoder(sparse=False, handle_unknown='ignore')
                                    ],
                    'tr__num__scal': [None,  StandardScaler()],
                    'ml__learning_rate': [0.03,0.02],
                    'ml__n_estimators' : [20,30]
    }]

In [25]:
# обучаем модель, используя логарифмирование зависимой 
train, y_train = cv_features[FTS_COLS], np.log1p(cv_features[TARGET])

In [26]:
%%time

rmsle = lambda y_true, y_pred: \
                np.sqrt(mean_squared_log_error(
                    y_true, pd.Series(y_pred).clip(lower=0)
                ))
if Phase_1:
    # создаем экземпляр класса RandomizedSearchCV или GridSearchCV, передав конвейер,
    # сетку гиперпараметров и указав количество
    # блоков перекрестной проверки, отключив запись метрик 
    # для обучающих блоков перекрестной проверки в атрибут cv_results_
    #gs = GridSearchCV(
    gs = RandomizedSearchCV( 
                      ml_pipe, 
                      param_grid, 
                      cv=tscv,   # CV, 
                      scoring=make_scorer(rmsle, greater_is_better=False), #True), #False),#
                      n_jobs=6, #-1, 
                      return_train_score=False,
                      verbose=10, #3,
                      random_state=SEED,
                    n_iter = 100
                     )
    # выполняем решетчатый поиск
    gs.fit(train, y_train)
    
    
    # записываем оптимальные значения гиперпараметров
    OPT_PARAMS_PIPE = gs.best_params_ 
    display(gs.best_params_, gs.best_score_)

Wall time: 0 ns


In [27]:
if Phase_1:
    #y_probas = gs.predict_proba(test)
    # смотрим наилучшие значения гиперпараметров
    print('Наилучшие значения гиперпараметров: {}'.format(OPT_PARAMS_PIPE))
    # смотрим наилучшее значение SCORE
    print('Наилучшее значение SCORE: {:.3f}'.format(gs.best_score_))
    # смотрим значение AUC на тестовой выборке
    #print('Значение SCORE на тестовой выборке: {:.3f}'.format(
    #   roc_auc_score(y_test, y_probas[:, 1] )))

In [28]:
if Phase_1:
    # увеличиваем количество выводимых строк
    pd.set_option('display.max_rows', 300)

    # выводим результаты
    cv_results = pd.DataFrame(gs.cv_results_)[['mean_test_score',
                                               'param_ml__n_estimators',
                                               'param_ml__learning_rate',
                                               'param_tr__cat__enc',
                                               'param_tr__num__scal'
                                               ]]
    cv_results = cv_results.sort_values(by='mean_test_score', ascending=False).reset_index(drop=True)

    for col in cv_results.columns[cv_results.columns.str.contains('enc')]:
        if col in 'mean_test_score':
            continue

        cv_results[col] = cv_results[col].apply(lambda x: type(x).__name__)

    display(cv_results)
    
    with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
        print('Временная метка окончания ФАЗЫ 1 : {}'.format(datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)
        print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_), file=logfile)
        print('Наилучшее значение SCORE: {:.3f}'.format(gs.best_score_), file=logfile)
        #print('Значение AUC на тестовой выборке: {:.3f}'.format(
        #    roc_auc_score(y_test, gs.predict_proba(test)[:, 1])), file=logfile)
        print('Таблица значений: ', file=logfile)
        #print(display(cv_results), file=logfile)
    with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
        cv_results.to_csv(NAME_RUN_LOG, index=False, mode='at')

> # ********  Phase 2 *************

In [29]:
if Phase_2 or Phase_3 or Phase_4:
    # трансформеры - ставим сюда лучший 
    # из найденных в 1-й фазе
    upd_cat_pipe = Pipeline([('enc',CatBoostEncoder() )])  
    upd_num_pipe = Pipeline([('scal', None)]) # 

    upd_transformers = [('num', upd_num_pipe, num_columns),
                        ('cat', upd_cat_pipe, cat_columns),]

    # передаем список трансформеров в ColumnTransformer
    upd_transformer = ColumnTransformer(transformers=upd_transformers)

    # задаем итоговый конвейер ПОКА ЧТО С НАЧАЛЬНЫМИ НАСТРОЙКАМИ
    ml_pipe2 = Pipeline([('tr', upd_transformer), 
                         ('ml', xgb.XGBRegressor(**params) )])

    # УСТАНОВКА В НОВЫЙ PIPE НАИЛУЧШИХ ЗНАЧЕНИЙ НАЙДЕННЫХ В ПРЕДЫДУЩЕЙ ФАЗЕ:  
    if Phase_1:  
        # присваиваем итоговому конвейеру оптимальные значения гиперпараметров
        print('Before modification ml__n_estimators ml_pipe2: ', ml_pipe2.get_params()['ml__n_estimators']) # 'n_estimators'
        print('Before modification ml__learning_rate ml_pipe2: ', ml_pipe2.get_params()['ml__learning_rate']) # 'n_estimators'
        ml_pipe2.set_params(**OPT_PARAMS_PIPE)
        print('After modification ml__n_estimators ml_pipe2: ', ml_pipe2.get_params()['ml__n_estimators'])
        print('After modification ml__learning_rate ml_pipe2: ', ml_pipe2.get_params()['ml__learning_rate'])

In [30]:
params2 = params3 = params4 = {}

if Phase_2: 
    params2 = {
        #'ml__use_label_encoder' : False ,
        # (1) гиперпараметры, задающие структуру дерева:
        #'tree_method' : 'gpu_hist' , 
        'ml__max_depth' : IntUniformDistribution(1 , 20),
        'ml__min_child_weight' : IntUniformDistribution(1 , 300),
        }

In [31]:
 # (2) гиперпараметры, привносящие случайность в ходе построения дерева:
if Phase_3: 
    params3 = {
              'ml__colsample_bytree' : UniformDistribution( 0 , 1.0),
              'ml__subsample' : UniformDistribution( 0 , 1.0)
             }

In [32]:
# (3) гиперпараметры, задающие регуляризацию функции потерь:
if Phase_4: 
    params4 = {
            'ml__lambda' : UniformDistribution( 1e-5 , 1.0),
            'ml__alpha' : UniformDistribution( 1e-5 , 1.0),
            'ml__gamma' : UniformDistribution( 1e-5 , 1.0)
             }

In [33]:
if Phase_2 or Phase_3 or Phase_4:  
    # задаем новое пространство поиска
    param_distributions = {**params2,**params3,**params4}
    print(param_distributions)

In [34]:
if Phase_2 or Phase_3 or Phase_4:  
    # задаем новое пространство поиска
    #param_distributions = {**params2,**params3,**params4}
 
    if TEST:
        param_distributions = {'ml__max_depth' : IntUniformDistribution(11, 13),
                              'ml__min_child_weight' : IntUniformDistribution(23, 25),}  

    # создаем экземпляр класса OptunaSearchCV
    optuna_search = OptunaSearchCV(
        ml_pipe2,
        param_distributions,
        scoring=make_scorer(rmsle, greater_is_better=False),
        n_trials=N_TRIALS,
        n_jobs=-1,
        refit=True,
        verbose=1,
        cv=tscv,   # CV_O,
        random_state=SEED)

In [35]:
%%time
if Phase_2 or Phase_3 or Phase_4:  
    # выполняем оптимизацию
    optuna_search.fit(train, y_train)

Wall time: 0 ns


In [36]:
if Phase_2 or Phase_3 or Phase_4:  
    # записываем оптимальные значения гиперпараметров optuna
    opt_optuna_params_pipe = optuna_search.best_params_
    if Phase_1:
        opt_optuna_params_pipe['ml__learning_rate'] = OPT_PARAMS_PIPE['ml__learning_rate']
        opt_optuna_params_pipe['ml__n_estimators'] = OPT_PARAMS_PIPE['ml__n_estimators']

    # печатаем наилучшие значения гиперпараметров
    print('Наилучшие значения гиперпараметров: {}'.format(opt_optuna_params_pipe))
    # печатаем наилучшее значение SCORE
    print('Наилучшее значение SCORE: {:.3f}'.format(optuna_search.best_score_))
    # смотрим значение SCORE на тестовой выборке
    #print('SCORE на тестовой выборке: {:.3f}'.format(
    #    rsmle(y_test, optuna_search.predict_proba(test)[:, 1])))
    print('')
    with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
        print('/nВременная метка окончания ФАЗЫ 2 : {}'.format(datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)
        print('Наилучшие значения гиперпараметров найденых optuna: \n{}'.format(opt_optuna_params_pipe), file=logfile)
        print('Наилучшее значение SCORE: {:.3f}'.format(optuna_search.best_score_), file=logfile)
        #print('SCORE на тестовой выборке: {:.3f}\n'.format(
        #    rsmle(y_test, optuna_search.predict_proba(test)[:, 1])), file=logfile)

> # ********  Final phase *************

In [37]:
# high parameters at the end
if HIGH_PARAM_END:
        opt_optuna_params_pipe['ml__learning_rate'] = 20000
        opt_optuna_params_pipe['ml__n_estimators'] = 0.0021

In [38]:
# формируем полный массив меток и массив признаков
# формируем полный массив тренировочных данных
# формируем полный массив меток и массив признаков   

In [39]:
for c in full_features2.columns:
    col_type = full_features2[c].dtype
    if col_type == 'object': # or col_type == 'int64': # or col_type.name == 'category':
        full_features2[c] = full_features2[c].astype('category')
full_features2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17879 entries, 0 to 17878
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   material_code  17879 non-null  int64   
 1   company_code   17879 non-null  int64   
 2   country        17879 non-null  category
 3   region         17879 non-null  category
 4   manager_code   17879 non-null  int64   
 5   month          17879 non-null  int64   
 6   vol_tm6        17879 non-null  float64 
 7   vol_tm5        17879 non-null  float64 
 8   vol_tm4        17879 non-null  float64 
 9   vol_tm3        17879 non-null  float64 
 10  vol_tm2        17879 non-null  float64 
 11  vol_tm1        17879 non-null  float64 
 12  last_year_avg  17879 non-null  float64 
 13  mdad_2         17879 non-null  float64 
 14  target         17879 non-null  float64 
dtypes: category(2), float64(9), int64(4)
memory usage: 1.8 MB


In [40]:
fulldata, y_fulldata = full_features2[FTS_COLS]  , np.log1p(full_features2[TARGET])

In [41]:
# трансформеры
final_cat_pipe = Pipeline([('enc',CatBoostEncoder() )])  #OneHotEncoder(handle_unknown='ignore', sparse=False))])
final_num_pipe = Pipeline([('scal', None)]) # 

final_transformers = [('num', final_num_pipe, num_columns),
                     ('cat', final_cat_pipe, cat_columns),]

# передаем список трансформеров в ColumnTransformer
final_transformer = ColumnTransformer(transformers=final_transformers)

# задаем итоговый конвейер ПОКА ЧТО С НАЧАЛЬНЫМИ НАСТРОЙКАМИ
final_ml_pipe = Pipeline([('tr', final_transformer), 
                     ('ml', xgb.XGBRegressor(**params) )])

if Phase_1:
    final_ml_pipe.set_params(**OPT_PARAMS_PIPE)


if Phase_2 or Phase_3 or Phase_4:
    # Для контроля изменений смотрим выбранные параметры модели 
    opt_par = ['ml__n_estimators','ml__learning_rate',
               'ml__max_depth','ml__min_child_weight','ml__subsample']
    print(opt_par)
    print( 'Before modification ml_pipe3: ' ,[final_ml_pipe.get_params()[x] for x in opt_par])
    
    # УСТАНОВКА В НОВЫЙ PIPE НАИЛУЧШИХ ЗНАЧЕНИЙ НАЙДЕННЫХ В ПРЕДЫДУЩИХ ФАЗАХ:  
    # присваиваем итоговому конвейеру оптимальные значения гиперпараметров найденные optuna
    final_ml_pipe.set_params(**opt_optuna_params_pipe)
    
    # Для контроля изменений
    print( 'After modification ml_pipe3: ' ,[final_ml_pipe.get_params()[x] for x in opt_par])

In [42]:
opt_par = ['ml__n_estimators','ml__learning_rate','ml__max_depth',
           'ml__min_child_weight','ml__colsample_bytree','ml__subsample',
           'ml__lambda','ml__alpha','ml__gamma']

for n, p in zip(opt_par,[final_ml_pipe.get_params()[x] for x in opt_par]):
    print(n, " : ", p)

ml__n_estimators  :  330
ml__learning_rate  :  0.055
ml__max_depth  :  4
ml__min_child_weight  :  115
ml__colsample_bytree  :  0.7359281520613952
ml__subsample  :  0.3350726311191104
ml__lambda  :  0.07017450086376546
ml__alpha  :  0.06778367298243107
ml__gamma  :  0.30129207712879835


In [43]:
with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
        print('Параметры конечной модели, формирующей посылку(submission): \n{}'.format(final_ml_pipe.get_params()), file=logfile)

In [44]:
# получаем ответы (вероятности) для проверочных блоков перекрестной проверки
XGBM_pred = cross_val_predict(final_ml_pipe, fulldata[-941*3:], y_fulldata[-941*3:], cv=10, n_jobs=-1, method='predict') #[:,1]
# смотрим значение rsmle
print('SCORE на всей исторической выборке: {:.3f}'.format(
        rmsle(y_fulldata[-941*3:], np.expm1(XGBM_pred) )))

SCORE на всей исторической выборке: 2.274


In [45]:
end_time = time()
time_taken = end_time - start_time # time_taken is in seconds
hours, rest = divmod(time_taken,3600)
minutes, seconds = divmod(rest, 60)
print('Duration: {} hours {} minutes {} seconds.'.format(hours, minutes, seconds))
print('\n\nВремя окончания работы скрипта: {}\n***********************\n'.format(datetime.now().strftime("%d-%m-%Y %H:%M")))
with open(NAME_RUN_LOG, 'at', encoding='utf-8') as logfile:
    print('\n\nВремя окончания работы скрипта: {}\n***********************\n'.format(datetime.now().strftime("%d-%m-%Y %H:%M")), 
          file=logfile)

Duration: 0.0 hours 0.0 minutes 46.937235832214355 seconds.


Время окончания работы скрипта: 17-12-2021 19:22
***********************



# Для использование в дальнейшем стекинге:

In [46]:
#XGBM_probs.to_csv("XGBM_probs_for_staking(v1)_TEST"+'.'+d_ext)
np.savetxt(NAME_OUTPUT_FOR_STAKING, XGBM_pred )
# Проверим
new_num_arr2 = np.loadtxt(NAME_OUTPUT_FOR_STAKING)
new_num_arr2[:10]

array([0.30203128, 4.38606739, 3.26788712, 1.57490647, 2.11707187,
       1.41892838, 5.43208838, 5.33820248, 3.30792236, 5.54022026])

# ### Adjusting over the entire historical sample Final phase

In [47]:
%%time

# обучаем обновленную модель на всей обучающей выборке
final_ml_pipe.fit(fulldata, y_fulldata)

Wall time: 2.68 s


Pipeline(steps=[('tr',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scal',
                                                                   None)]),
                                                  ['vol_tm6', 'vol_tm5',
                                                   'vol_tm4', 'vol_tm3',
                                                   'vol_tm2', 'vol_tm1',
                                                   'last_year_avg', 'mdad_2']),
                                                 ('cat',
                                                  Pipeline(steps=[('enc',
                                                                   CatBoostEncoder())]),
                                                  ['material_code',
                                                   'company_code', 'country',
                                                   'region', 'manager_code',
                           

In [48]:
dill.dump(final_ml_pipe, file = open("pipe_XGB_1.cbm", "wb"))

In [49]:
%%writefile predict.py

import pathlib
import pandas as pd
import numpy as np
#from catboost import CatBoostRegressor
#import lightgbm as lgb
#import xgboost as xgb
#from sklearn.pipeline import Pipeline
#from sklearn.compose import ColumnTransformer
import dill
#from category_encoders import TargetEncoder, OrdinalEncoder, CatBoostEncoder

MODEL_FILE = pathlib.Path(__file__).parent.joinpath("pipe_XGB_1.cbm")
#ENCODER_1_FILE = pathlib.Path(__file__).parent.joinpath("ET_encoder_1.pkl")
#ENCODER_2_FILE = pathlib.Path(__file__).parent.joinpath("ET_encoder_2.pkl")

AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]
CAT_COLS = ["material_code", "company_code", "country", "region", "manager_code", "month"]
FTS_COLS = ['material_code', 'company_code', 'country', 'region', 'manager_code', 'month', 
            'vol_tm6', 'vol_tm5', 'vol_tm4', 'vol_tm3', 'vol_tm2', 'vol_tm1', 'last_year_avg', 'mdad_2']

TARGET = "target"

def get_features(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:
    """Вычисление признаков для `month`."""

    start_period = month - pd.offsets.MonthBegin(6)
    end_period = month - pd.offsets.MonthBegin(1)

    df = df.loc[:, :end_period]

    features = pd.DataFrame([], index=df.index)
    features["month"] = month.month
    features[[f"vol_tm{i}" for i in range(6, 0, -1)]] = df.loc[:, start_period:end_period].copy()

   ### !!!!!!!!!!!!!!   #################################################################################
    rolling = df.rolling(12, axis=1, min_periods=1)
    features = features.join(rolling.mean().iloc[:, -1].rename("last_year_avg"))
    
    # Добавление скользящих средних абсолютные отклонения(MDAD)
    rolling = df.rolling(2, axis=1, min_periods=1)
    features = features.join( 
                    rolling.apply(lambda x: np.nanmedian(np.abs(x - np.nanmedian(x))) , raw=True 
                    ).iloc[:, -1].rename("mdad_2") )
    
    return features.reset_index()


def predict(df: pd.DataFrame, month: pd.Timestamp) -> pd.DataFrame:

    model = dill.load(open(MODEL_FILE, "rb"))

    
    
    group_ts = df.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
    features = get_features(group_ts, month)
    for c in features.columns:
        col_type = features[c].dtype
        if col_type == 'object': 
            features[c] = features[c].astype('category')    
    
    predictions = np.expm1(model.predict(features[FTS_COLS]))
    predictions = pd.Series(predictions).clip(lower=0, upper=7000)

    preds_df = features[AGG_COLS].copy()
    preds_df["prediction"] = predictions
    return preds_df

Overwriting predict.py


In [50]:
import predict
import importlib
importlib.reload(predict)

<module 'predict' from 'C:\\Users\\cvirk\\PROJECTS\\Соревнования\\Sibur2021\\predict.py'>

In [51]:
predict.get_features(group_ts.iloc[:, :-1], pd.Timestamp("2020-07-01"))

,material_code,company_code,country,region,manager_code,month,vol_tm6,vol_tm5,vol_tm4,vol_tm3,vol_tm2,vol_tm1,last_year_avg,mdad_2
0,124,7278,Россия,Респ. Татарстан,17460,7,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,133,0,Белоруссия,Минская обл.,10942,7,17.0,72.0,250.0,394.0,288.0,210.0,179.833333,39.0
2,133,0,Белоруссия,Могилевская обл.,10942,7,0.0,0.0,0.0,166.0,62.0,0.0,19.000000,31.0
3,133,0,Белоруссия,г. Минск,10942,7,0.0,0.0,0.0,21.0,0.0,0.0,22.416667,0.0
4,133,0,Казахстан,г. Нур-Султан,13301,7,0.0,50.0,40.0,0.0,0.0,50.0,30.000000,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,7,0.0,21.0,63.0,125.0,84.0,84.0,31.416667,0.0
937,998,0,Россия,Ленинградская обл.,18079,7,5.0,5.0,5.0,0.0,3.0,3.0,4.750000,0.0
938,998,3380,Россия,Ленинградская обл.,14956,7,73.0,74.0,122.0,100.0,15.0,30.0,85.583333,7.5
939,998,5410,Россия,г. Санкт-Петербург,14956,7,100.0,180.0,180.0,100.0,140.0,40.0,118.333333,50.0


In [52]:
ts_preds_tst = predict.predict(data[data.month<"2020-07-01"], pd.Timestamp("2020-07-01"))
ts_preds_tst 

,material_code,company_code,country,region,manager_code,prediction
0,124,7278,Россия,Респ. Татарстан,17460,0.785872
1,133,0,Белоруссия,Минская обл.,10942,113.658333
2,133,0,Белоруссия,Могилевская обл.,10942,5.807093
3,133,0,Белоруссия,г. Минск,10942,3.148300
4,133,0,Казахстан,г. Нур-Султан,13301,13.913647
...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,49.378426
937,998,0,Россия,Ленинградская обл.,18079,1.375150
938,998,3380,Россия,Ленинградская обл.,14956,26.083757
939,998,5410,Россия,г. Санкт-Петербург,14956,44.128330


In [53]:
print("Ошибка на тестовом множестве:",
      f'{np.sqrt(mean_squared_log_error(group_ts.reset_index().iloc[:,-1], ts_preds_tst["prediction"])):.4f}')

Ошибка на тестовом множестве: 1.4954


In [54]:
ts_range = pd.date_range("2019-07-01", "2020-07-01", freq="MS")
s_csore = 0
for cnt, m in enumerate(ts_range):
    #print(cnt)
    ts_preds_ = predict.predict(data[data.month<"2020-07-01"], pd.Timestamp(m))
    #print(ts_preds_["prediction"])
    #print(group_ts.reset_index().loc[:,m])
    score = np.sqrt(mean_squared_log_error(group_ts.reset_index().loc[:,m], ts_preds_["prediction"]))
    print(m, " -> Ошибка на тестовом множестве:",
      f'{score:.4f}'
         )
    s_csore += score

print("Суммарная ошибка на тестовом множестве:",
      f'{s_csore/len(ts_range):.4f}'
         )

2019-07-01 00:00:00  -> Ошибка на тестовом множестве: 1.5434
2019-08-01 00:00:00  -> Ошибка на тестовом множестве: 1.4496
2019-09-01 00:00:00  -> Ошибка на тестовом множестве: 1.4524
2019-10-01 00:00:00  -> Ошибка на тестовом множестве: 1.4441
2019-11-01 00:00:00  -> Ошибка на тестовом множестве: 1.5381
2019-12-01 00:00:00  -> Ошибка на тестовом множестве: 1.5795
2020-01-01 00:00:00  -> Ошибка на тестовом множестве: 1.6819
2020-02-01 00:00:00  -> Ошибка на тестовом множестве: 1.6496
2020-03-01 00:00:00  -> Ошибка на тестовом множестве: 1.5996
2020-04-01 00:00:00  -> Ошибка на тестовом множестве: 1.7565
2020-05-01 00:00:00  -> Ошибка на тестовом множестве: 1.6624
2020-06-01 00:00:00  -> Ошибка на тестовом множестве: 1.7055
2020-07-01 00:00:00  -> Ошибка на тестовом множестве: 1.4954
Суммарная ошибка на тестовом множестве: 1.5814


In [55]:
# Windows
!tar.exe -a -c -f Pip_XGB_7.zip pipe_XGB_1.cbm requirements.txt predict.py
!wsl ls -al | wsl grep Pip_XGB_

-rwxrwxrwx 1 dimacv dimacv     55213 Dec 17 13:51 Pip_XGB_1.zip
-rwxrwxrwx 1 dimacv dimacv     88657 Dec 17 13:57 Pip_XGB_2.zip
-rwxrwxrwx 1 dimacv dimacv     88690 Dec 17 14:04 Pip_XGB_3.zip
-rwxrwxrwx 1 dimacv dimacv     88690 Dec 17 14:16 Pip_XGB_4.zip
-rwxrwxrwx 1 dimacv dimacv    123136 Dec 17 19:17 Pip_XGB_5.zip
-rwxrwxrwx 1 dimacv dimacv    123136 Dec 17 19:23 Pip_XGB_7.zip
